# Install Dependecies Library

In [ ]:
!pip install -r requirements.txt

# Import Library

In [3]:
from ultralytics import YOLO
import supervision as sv
import numpy as np
from tqdm.notebook import tqdm

# Load Model

In [4]:
model = YOLO("yolov8x.pt")
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 221MB/s]


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


# Load Video

In [5]:
# directory video
SOURCE_VIDEO_PATH = "assets/toll_gate.mp4"

# directory to saving video result detection and count
RESULT_VIDEO_PATH = "assets/toll_gate-result.mp4"

# Detect and Counting Vehicle

In [6]:
CLASS_NAMES_DICT = model.model.names
selected_classes = [2,5] # car,bus

# line for counting vehicle
LINE_START = sv.Point(10, 155)
LINE_END = sv.Point(560, 275)

In [ ]:
# read video frame by frame
video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)
generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)

# load tracker using byte_track
tracker = sv.ByteTrack()
line_zone = sv.LineZone(start=LINE_START, end=LINE_END,
                        triggering_anchors=[sv.Position.TOP_RIGHT])

box_annotator = sv.BoxAnnotator() # bounding box
trace_annotator = sv.TraceAnnotator() # tracer vehicle

# annotate line and text
line_zone_annotator = sv.LineZoneAnnotator(color=sv.Color.YELLOW,
                                           text_thickness=1, text_offset=14,
                                           text_scale=0.6, text_padding=15,
                                           custom_in_text="Vehicle Numbers",
                                           display_out_count=False)

# annotate label detection
label_annotator = sv.LabelAnnotator(text_color=sv.Color.RED,
                                    border_radius=3)

with sv.VideoSink(target_path=RESULT_VIDEO_PATH, video_info=video_info) as sink:
  for frame in tqdm(generator, total=video_info.total_frames):
    results = model.track(frame, persist=True, classes=[selected_classes])[0] # predict

    detections = sv.Detections.from_ultralytics(results) # convert to supervision
    detections = detections[np.isin(detections.class_id, selected_classes)] # filter class
    detections = tracker.update_with_detections(detections) # update tracker
    line_zone.trigger(detections) # trigger line

    # format custom labels
    labels = [
        f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
        for tracker_id, confidence, class_id in zip(detections.tracker_id, detections.confidence, detections.class_id)
    ]

    annotated_frame = trace_annotator.annotate(scene=frame.copy(),detections=detections) # tracer vehicle
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections) # bounding box
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels) # label detection
    annotated_frame = line_zone_annotator.annotate(annotated_frame, line_counter=line_zone) # annotate line and text

    sink.write_frame(annotated_frame) # write video result